## Environment setup

In [1]:
import glob, os, time
from datetime import datetime
from functools import partial
import importlib

import numpy as np

import tensorflow as tf

## Networks definitions and adaptations

In [2]:
from architecture_definitions import *

dir_postfix = get_classic_nn_dir_postfix() 
    
print_Classic_NN()  

Classic NN definitions:
dense_layer1_size: 128
dense_layer2_size: 64
dense_layer3_size: 32
dense_layer4_size: 1
------------------------


### Training data set preparation

In [3]:
import io_functions as io
importlib.reload(io)

batchSize = 4096
nEpochs = 1

#trainDataDir = "/scratch_ssd/akalinow/ProgrammingProjects/MachineLearning/OMTF/data/18_12_2020/"   
trainDataDir = "/home/kbunkow/cms_data/OMTF_data_2020/18_12_2020/"
trainFileNames = glob.glob(trainDataDir+'OMTFHits_pats0x0003_oldSample_files_*_chunk_0.tfrecord.gzip')

trainDataDir = "/scratch_cmsse/alibordi/data/training/"
trainFileNames = glob.glob(trainDataDir+'*OneOverPt*tfrecord.gzip')

dataset = io.get_Classic_NN_dataset(batchSize, nEpochs, trainFileNames, isTrain=True)

print("dataset", dataset)

Reading data from files:
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_April4_chunk_0_filtered.tfrecord.gzip
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_Feb15_chunk_0_filtered.tfrecord.gzip
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_Feb22_chunk_0_filtered.tfrecord.gzip
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_April14_chunk_0_filtered.tfrecord.gzip


2023-04-20 07:20:10.804431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-20 07:20:10.814471: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-20 07:20:10.814784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-20 07:20:10.816579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

dataset <ParallelMapDataset element_spec=(TensorSpec(shape=(4096, 37), dtype=tf.float16, name=None), (TensorSpec(shape=(4096,), dtype=tf.float16, name=None),), TensorSpec(shape=(4096,), dtype=tf.float16, name=None))>


### Model definition

In [4]:
import model_functions as models
importlib.reload(models)

import io_functions as io
importlib.reload(io)

networkInputSize = 2 * np.sum(io.getFeaturesMask()) + 1
loss_fn = 'mape'
loss_fn = loss_MAPE_MAE

model = models.get_Classic_NN(networkInputSize=networkInputSize, loss_fn=loss_fn)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pt_layer_1 (Dense)          (None, 128)               4864      
                                                                 
 pt_layer_2 (Dense)          (None, 64)                8256      
                                                                 
 pt_layer_3 (Dense)          (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 15,233
Trainable params: 15,233
Non-trainable params: 0
_________________________________________________________________


### The training loop

In [5]:
from keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2070 SUPER, compute capability 7.5
Compute dtype: float16
Variable dtype: float32


2023-04-20 07:20:12.052659: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [6]:
%%time
current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training start. Current Time =", current_time)

nEpochs = 150

log_dir = "logs/fit/" + current_time + dir_postfix
job_dir = "training/" + current_time + dir_postfix

checkpoint_path = job_dir + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq = 5085)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=(10, 20))
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=5, verbose=1)
callbacks =  [tensorboard_callback, cp_callback, early_stop_callback]

model.save_weights(checkpoint_path.format(epoch=0))
   
model.fit(dataset.skip(10), 
          epochs=nEpochs, shuffle=True,
          callbacks=callbacks,
          validation_data = dataset.take(10)
            )
model.save(job_dir, save_format='tf')

current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training end. Current Time =", current_time)

Training start. Current Time = 2023_Apr_20_07_20_12


2023-04-20 07:20:12.066990: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2023-04-20 07:20:12.067012: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2023-04-20 07:20:12.067045: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2023-04-20 07:20:12.220357: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2023-04-20 07:20:12.221599: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed


Epoch 1/150
     10/Unknown - 1s 21ms/step - loss: 2203.9333

2023-04-20 07:20:13.280801: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2023-04-20 07:20:13.280829: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.


     18/Unknown - 1s 18ms/step - loss: 1350.9750

2023-04-20 07:20:13.626779: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2023-04-20 07:20:13.638479: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2023-04-20 07:20:13.964183: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 1671 callback api events and 1650 activity events. 
2023-04-20 07:20:14.236445: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2023-04-20 07:20:14.882529: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/2023_Apr_20_07_20_12_classic_128_64_32_1/plugins/profile/2023_04_20_07_20_14

2023-04-20 07:20:15.660221: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/fit/2023_Apr_20_07_20_12_classic_128_64_32_1/plugins/profile/2023_04_20_07_20_14/fba800be2533.trace.json.gz


     30/Unknown - 4s 98ms/step - loss: 916.1741 

2023-04-20 07:20:15.920378: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/2023_Apr_20_07_20_12_classic_128_64_32_1/plugins/profile/2023_04_20_07_20_14

2023-04-20 07:20:15.925525: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/fit/2023_Apr_20_07_20_12_classic_128_64_32_1/plugins/profile/2023_04_20_07_20_14/fba800be2533.memory_profile.json.gz
2023-04-20 07:20:15.931707: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/fit/2023_Apr_20_07_20_12_classic_128_64_32_1/plugins/profile/2023_04_20_07_20_14
Dumped tool data for xplane.pb to logs/fit/2023_Apr_20_07_20_12_classic_128_64_32_1/plugins/profile/2023_04_20_07_20_14/fba800be2533.xplane.pb
Dumped tool data for overview_page.pb to logs/fit/2023_Apr_20_07_20_12_classic_128_64_32_1/plugins/profile/2023_04_20_07_20_14/fba800be2533.overview_page.pb
Dumped tool data for input_pipeline.pb to logs

787/787 [==============================] - 22s 27ms/step - loss: 164.5494 - val_loss: 90.8507
Epoch 2/150
787/787 [==============================] - 17s 21ms/step - loss: 97.4770 - val_loss: 97.9156
Epoch 3/150
787/787 [==============================] - 17s 22ms/step - loss: 89.8830 - val_loss: 80.0709
Epoch 4/150
787/787 [==============================] - 17s 22ms/step - loss: 85.2027 - val_loss: 82.9249
Epoch 5/150
787/787 [==============================] - 17s 21ms/step - loss: 81.6556 - val_loss: 77.5365
Epoch 6/150
787/787 [==============================] - 17s 22ms/step - loss: 78.7384 - val_loss: 77.3029
Epoch 7/150
360/787 [============>.................] - ETA: 9s - loss: 77.3541
Epoch 7: saving model to training/2023_Apr_20_07_20_12_classic_128_64_32_1/cp-0007.ckpt
787/787 [==============================] - 17s 22ms/step - loss: 76.4882 - val_loss: 73.6245
Epoch 8/150
787/787 [==============================] - 17s 22ms/step - loss: 74.4335 - val_loss: 73.8082
Epoch 9/150
787/